<a href="https://colab.research.google.com/github/giobritos/soul_code_projeto_final/blob/main/Import_Originais_SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![São Paulo](https://storage.googleapis.com/projeto-final-agsw/Imagens/transparencia_ssp.png)

# 👮 **Importação das Bases Originais de São Paulo**

Os dados escolhidos estão disponíveis no **Portal do Governo do estado de São Paulo**, que fica localizado no **Brasil**. [Neste link](https://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx) você acessa eles para consulta e download publicamente. 

Eles representam os boletins de ocorrência registrados no estado, ao qual vamos filtrar por registros que tiveram **homicidios** dos anos de **2018** à **2021**.

Link para a documentação completa do projeto: [clique aqui](https://www.notion.so/Mindful-Data-Consultoria-d495964059c34acb9c8fe3d21dbb5cf6)

## **📚 1 - Instalação e importação das bibliotecas**

In [ ]:
# Instaladores 

# Google Cloud Storage
%%capture
%pip install gcsfs

# Conector MySQL
%pip install mysql-connector-python
%pip install PyMySQL

In [ ]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy

In [ ]:
# Conector do Mysql
from google.cloud.sql.connector import Connector
import sqlalchemy

# Conexão do Storage
from google.cloud import storage

# Importa sistema operacional
import os

# Pandas
import pandas as pd

# Numpy
import numpy as np

# setando configurações de display do pandas
pd.set_option('display.max_columns',100)

## 🔗 **2 - Conexão com o DataLake**

In [ ]:
# Configuração da chave de segurança
serviceAccount = '/content/projeto-final-373521-25961e56ca37.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

# Fazer acesso ao bucket do Cloud Storage
client = storage.Client()

# Criar uma variável para receber o nome do bucket
bucket = client.get_bucket('projeto-final-agsw')

## ⚒️📑 **3 - Extração e padronização das bases de dados**

Os dados estão no formato .xlsx (excel), separados anualmente em cada aba, vamos juntar todos em uma só tabela para fins de centralização e organização dos dados.

### ☑️ **Dados de homicidios dolosos**



In [ ]:
# Gerando Data Frames
dfhom1 = pd.read_excel('gs://datasets-originais-agsw/ssp/Homicidio_2017_2022.xlsx', sheet_name='2018')
dfhom2 = pd.read_excel('gs://datasets-originais-agsw/ssp/Homicidio_2017_2022.xlsx', sheet_name='2019')
dfhom3 = pd.read_excel('gs://datasets-originais-agsw/ssp/Homicidio_2017_2022.xlsx', sheet_name='2020')
dfhom4 = pd.read_excel('gs://datasets-originais-agsw/ssp/Homicidio_2017_2022.xlsx', sheet_name='2021')

In [ ]:
# Concatenando os Data Frames
dfsp_homicidio = pd.concat([dfhom1,dfhom2,dfhom3,dfhom4])

In [ ]:
# Visualizando estrutura
dfsp_homicidio.head(1)

,DEPARTAMENTO_CIRCUNSCRICAO,SECCIONAL_CIRCUNSCRICAO,MUNICIPIO_CIRCUNSCRICAO,DP_CIRCUNSCRICAO,HD,Nº DE VÍT HD,ID_DELEGACIA,MÊS ESTATISTICA,ANO ESTATISTICA,DATAHORA_REGISTRO_BO,NUM_BO,ANO_BO,MUNICIPIO_ELABORACAO,DP_ELABORACAO,SEC_ELABORACAO,DEP_ELABORACAO,DATA_FATO,HORA_FATO,DESC_TIPOLOCAL,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,TIPO_PESSOA,SEXO_PESSOA,IDADE_PESSOA,DATA_NASCIMENTO_PESSOA,COR_PELE,PROFISSAO,NATUREZA_APURADA
0,DECAP,1ª SEC,São Paulo,002 DP - Bom Retiro,1.0,1,200100,1,2018,2018-01-07,16,2018,S.PAULO,DHPP DIV. HOMICIDIOS,DHPP - DEP. HOM. PROTECAO PESSOA,DHPP,2018-01-06,20:06:00,Via pública,AVENIDA PRESIDENTE CASTELO BRANCO,6173.0,-23.523712,-46.606388,Vítima,Masculino,NaN,NaT,Ignorada,NaN,HOMICÍDIO DOLOSO


In [ ]:
dfsp_homicidio.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO ', 'HD', 'Nº DE VÍT HD',
       'ID_DELEGACIA', 'MÊS ESTATISTICA', 'ANO ESTATISTICA',
       'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO', 'MUNICIPIO_ELABORACAO',
       'DP_ELABORACAO', 'SEC_ELABORACAO', 'DEP_ELABORACAO', 'DATA_FATO',
       'HORA_FATO', 'DESC_TIPOLOCAL', 'LOGRADOURO', 'NUMERO_LOGRADOURO',
       'LATITUDE', 'LONGITUDE', 'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA',
       'DATA_NASCIMENTO_PESSOA', 'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
# Realizando a troca de algumas colunas
dfsp_homicidio.rename(columns={'DP_CIRCUNSCRICAO ':'DP_CIRCUNSCRICAO'}, inplace=True)

In [ ]:
# Verificando a quantidade de ocorrências por ano
dfsp_homicidio.groupby(['NATUREZA_APURADA', 'ANO ESTATISTICA'],dropna=False).size().sort_values(ascending=False)

NATUREZA_APURADA  ANO ESTATISTICA
HOMICÍDIO DOLOSO  2018               3106
                  2020               3038
                  2019               2906
                  2021               2847
dtype: int64

### ☑️ **Dados de Latrocinio**

In [ ]:
# Trazendo cada aba da planilha para o dataset
dfla1 = pd.read_excel('gs://datasets-originais-agsw/ssp/Latrocinio_2018_2022.xlsx', sheet_name='2018')
dfla2 = pd.read_excel('gs://datasets-originais-agsw/ssp/Latrocinio_2018_2022.xlsx', sheet_name='2019')
dfla3 = pd.read_excel('gs://datasets-originais-agsw/ssp/Latrocinio_2018_2022.xlsx', sheet_name='2020')
dfla4 = pd.read_excel('gs://datasets-originais-agsw/ssp/Latrocinio_2018_2022.xlsx', sheet_name='2021')

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Concatenando os Data Frames
dfsp_latrocinio = pd.concat([dfla1,dfla2,dfla3,dfla4])

In [ ]:
# Visualizando estrutura
dfsp_latrocinio.head(1)

,DEPARTAMENTO_CIRCUNSCRICAO,SECCIONAL_CIRCUNSCRICAO,MUNICIPIO_CIRCUNSCRICAO,DP_CIRCUNSCRICAO,LATRO,Nº DE VIT EM LATRO,ID_DELEGACIA,MÊS ESTATISTICA,ANO ESTATISTICA,DATAHORA_REGISTRO_BO,NUM_BO,ANO_BO,MUNICIPIO_ELABORACAO,DP_ELABORACAO,SEC_ELABORACAO,DEP_ELABORACAO,DATA_FATO,HORA_FATO,DESC_TIPOLOCAL,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,TIPO_PESSOA,SEXO_PESSOA,IDADE_PESSOA,DATA_NASCIMENTO_PESSOA,COR_PELE,PROFISSAO,NATUREZA_APURADA
0,DECAP,1ª SEC,São Paulo,005 DP - Aclimação,1.0,1,10342,2018,1,2018-01-21,550,2018,S.PAULO,78º D.P. JARDINS,DEL.SEC.1º CENTRO,DECAP,2018-01-21,00:02:00,Via pública,AVENIDA BRIGADEIRO LUIS ANTONIO,1572.0,-23.562435,-46.645048,Vítima,Masculino,59.0,1958-02-18,Branca,JORNALISTA,LATROCÍNIO


In [ ]:
dfsp_latrocinio.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO ', 'LATRO',
       'Nº DE VIT EM LATRO', 'ID_DELEGACIA', 'MÊS ESTATISTICA',
       'ANO ESTATISTICA', 'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO',
       'MUNICIPIO_ELABORACAO', 'DP_ELABORACAO', 'SEC_ELABORACAO',
       'DEP_ELABORACAO', 'DATA_FATO', 'HORA_FATO', 'DESC_TIPOLOCAL',
       'LOGRADOURO', 'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA', 'DATA_NASCIMENTO_PESSOA',
       'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
# Realizando a troca de algumas colunas
dfsp_latrocinio.rename(columns={'DP_CIRCUNSCRICAO ':'DP_CIRCUNSCRICAO'}, inplace=True)

In [ ]:
# Realizando a troca de algumas colunas
dfsp_latrocinio.rename(columns={'MÊS ESTATISTICA': 'ANO ESTATISTICA', 'ANO ESTATISTICA': 'MÊS ESTATISTICA' }, inplace=True)

In [ ]:
# Visualizando colunas
dfsp_latrocinio.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO', 'LATRO',
       'Nº DE VIT EM LATRO', 'ID_DELEGACIA', 'ANO ESTATISTICA',
       'MÊS ESTATISTICA', 'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO',
       'MUNICIPIO_ELABORACAO', 'DP_ELABORACAO', 'SEC_ELABORACAO',
       'DEP_ELABORACAO', 'DATA_FATO', 'HORA_FATO', 'DESC_TIPOLOCAL',
       'LOGRADOURO', 'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA', 'DATA_NASCIMENTO_PESSOA',
       'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
# Reorganizando Data Frame
dfsp_latrocinio = dfsp_latrocinio[['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO', 'LATRO',
       'Nº DE VIT EM LATRO', 'ID_DELEGACIA', 'MÊS ESTATISTICA','ANO ESTATISTICA',
        'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO',
       'MUNICIPIO_ELABORACAO', 'DP_ELABORACAO', 'SEC_ELABORACAO',
       'DEP_ELABORACAO', 'DATA_FATO', 'HORA_FATO', 'DESC_TIPOLOCAL',
       'LOGRADOURO', 'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA', 'DATA_NASCIMENTO_PESSOA',
       'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA']]

In [ ]:
# Visualizando estrutura
dfsp_latrocinio.head(1)

,DEPARTAMENTO_CIRCUNSCRICAO,SECCIONAL_CIRCUNSCRICAO,MUNICIPIO_CIRCUNSCRICAO,DP_CIRCUNSCRICAO,LATRO,Nº DE VIT EM LATRO,ID_DELEGACIA,MÊS ESTATISTICA,ANO ESTATISTICA,DATAHORA_REGISTRO_BO,NUM_BO,ANO_BO,MUNICIPIO_ELABORACAO,DP_ELABORACAO,SEC_ELABORACAO,DEP_ELABORACAO,DATA_FATO,HORA_FATO,DESC_TIPOLOCAL,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,TIPO_PESSOA,SEXO_PESSOA,IDADE_PESSOA,DATA_NASCIMENTO_PESSOA,COR_PELE,PROFISSAO,NATUREZA_APURADA
0,DECAP,1ª SEC,São Paulo,005 DP - Aclimação,1.0,1,10342,1,2018,2018-01-21,550,2018,S.PAULO,78º D.P. JARDINS,DEL.SEC.1º CENTRO,DECAP,2018-01-21,00:02:00,Via pública,AVENIDA BRIGADEIRO LUIS ANTONIO,1572.0,-23.562435,-46.645048,Vítima,Masculino,59.0,1958-02-18,Branca,JORNALISTA,LATROCÍNIO


In [ ]:
# Verificando a quantidade de ocorrências por ano
dfsp_latrocinio.groupby(['NATUREZA_APURADA', 'ANO ESTATISTICA'],dropna=False).size().sort_values(ascending=False)

NATUREZA_APURADA  ANO ESTATISTICA
LATROCÍNIO        2018               278
                  2019               199
                  2020               183
                  2021               173
dtype: int64

### ☑️ **Dados de Lesão Corporal seguida de Morte**

In [ ]:
# Trazendo cada aba da planilha para o dataset
dfle1 = pd.read_excel('gs://datasets-originais-agsw/ssp/LCSM_2016_2022.xlsx', sheet_name='2017')
dfle2 = pd.read_excel('gs://datasets-originais-agsw/ssp/LCSM_2016_2022.xlsx', sheet_name='2018')
dfle3 = pd.read_excel('gs://datasets-originais-agsw/ssp/LCSM_2016_2022.xlsx', sheet_name='2019')
dfle4 = pd.read_excel('gs://datasets-originais-agsw/ssp/LCSM_2016_2022.xlsx', sheet_name='2020')
dfle5 = pd.read_excel('gs://datasets-originais-agsw/ssp/LCSM_2016_2022.xlsx', sheet_name='2021')

In [ ]:
# Concatenando os Data Frames
dfsp_lesao = pd.concat([dfle1,dfle2,dfle3,dfle4, dfle5])

In [ ]:
dfsp_lesao.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO', 'LCSM', 'Unnamed: 5',
       'ID_DELEGACIA', 'MÊS ESTATISTICA', 'ANO ESTATISTICA',
       'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO', 'MUNICIPIO_ELABORACAO',
       'DP_ELABORACAO', 'SEC_ELABORACAO', 'DEP_ELABORACAO', 'DATA_FATO',
       'HORA_FATO', 'DESC_TIPOLOCAL', 'LOGRADOURO', 'NUMERO_LOGRADOURO',
       'LATITUDE', 'LONGITUDE', 'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA',
       'DATA_NASCIMENTO_PESSOA', 'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
# Realizando a troca de algumas colunas
dfsp_lesao.rename(columns={'DP_CIRCUNSCRICAO ':'DP_CIRCUNSCRICAO'}, inplace=True)

In [ ]:
# Visualizando estrutura
dfsp_lesao.head(1)

,DEPARTAMENTO_CIRCUNSCRICAO,SECCIONAL_CIRCUNSCRICAO,MUNICIPIO_CIRCUNSCRICAO,DP_CIRCUNSCRICAO,LCSM,Unnamed: 5,ID_DELEGACIA,MÊS ESTATISTICA,ANO ESTATISTICA,DATAHORA_REGISTRO_BO,NUM_BO,ANO_BO,MUNICIPIO_ELABORACAO,DP_ELABORACAO,SEC_ELABORACAO,DEP_ELABORACAO,DATA_FATO,HORA_FATO,DESC_TIPOLOCAL,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,TIPO_PESSOA,SEXO_PESSOA,IDADE_PESSOA,DATA_NASCIMENTO_PESSOA,COR_PELE,PROFISSAO,NATUREZA_APURADA
0,DECAP,5ª SEC,São Paulo,056 DP - Vila Alpina,1,NaN,10310,1,2017,2017-01-05 00:00:00,74,2017,S.PAULO,56º D.P. VILA ALPINA,DEL.SEC.5º LESTE,DECAP,2017-01-05 00:00:00,08:10:00,Saúde,AVENIDA FRANCISCO FALCONI,1501.0,NaN,NaN,Vítima,Feminino,32.0,1984-12-06 00:00:00,Parda,VENDEDOR(A),LESÃO CORPORAL SEGUIDA DE MORTE


### ☑️ **Dados de Morte decorrente de intervenção policial**

In [ ]:
# Trazendo cada aba da planilha para o dataset
dfmo1 = pd.read_excel('gs://datasets-originais-agsw/ssp/MDIP_2013_2022.xlsx', sheet_name='2017')
dfmo2 = pd.read_excel('gs://datasets-originais-agsw/ssp/MDIP_2013_2022.xlsx', sheet_name='2018')
dfmo3 = pd.read_excel('gs://datasets-originais-agsw/ssp/MDIP_2013_2022.xlsx', sheet_name='2019')
dfmo4 = pd.read_excel('gs://datasets-originais-agsw/ssp/MDIP_2013_2022.xlsx', sheet_name='2020')
dfmo5 = pd.read_excel('gs://datasets-originais-agsw/ssp/MDIP_2013_2022.xlsx', sheet_name='2021')

In [ ]:
# Concatenando os Data Frames
dfsp_policia = pd.concat([dfmo1,dfmo2,dfmo3,dfmo4, dfmo5])

In [ ]:
# Visualizando estrutura
dfsp_policia.head(1)

,DEPARTAMENTO_CIRCUNSCRICAO,SECCIONAL_CIRCUNSCRICAO,MUNICIPIO_CIRCUNSCRICAO,DP_CIRCUNSCRICAO,COORPORAÇÃO,SITUAÇÃO,ID_DELEGACIA,MÊS ESTATISTICA,ANO ESTATISTICA,DATAHORA_REGISTRO_BO,NUM_BO,ANO_BO,MUNICIPIO_ELABORACAO,DP_ELABORACAO,SEC_ELABORACAO,DEP_ELABORACAO,DATA_FATO,HORA_FATO,DESC_TIPOLOCAL,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,TIPO_PESSOA,SEXO_PESSOA,IDADE_PESSOA,DATA_NASCIMENTO_PESSOA,COR_PELE,PROFISSAO,NATUREZA_APURADA
0,DEMACRO,DEL.SEC.TABOÃO DA SERRA,ITAPECERICA DA SERRA,DEL.POL.ITAPECERICA DA SERRA,PM,folga,30406,1,2017,2017-01-01 00:00:00,1,2017,ITAPECERICA DA SERRA,DEL.POL.ITAPECERICA DA SERRA,DEL.SEC.TABOÃO DA SERRA,DEMACRO,2017-01-01 00:00:00,01:00:00,Via pública,RUA JULIO MANOEL DE ARAUJO,218,-23.703543,-46.80364,Autor/Vitima,Masculino,27.0,1989-01-07 00:00:00,Parda,GUARDA MUNICIPAL,MORTE DECORRENTE DE INTERVENÇÃO POLICIAL


In [ ]:
dfsp_policia.columns

Index(['DEPARTAMENTO_CIRCUNSCRICAO', 'SECCIONAL_CIRCUNSCRICAO',
       'MUNICIPIO_CIRCUNSCRICAO', 'DP_CIRCUNSCRICAO ', 'COORPORAÇÃO',
       'SITUAÇÃO', 'ID_DELEGACIA', 'MÊS ESTATISTICA', 'ANO ESTATISTICA',
       'DATAHORA_REGISTRO_BO', 'NUM_BO', 'ANO_BO', 'MUNICIPIO_ELABORACAO',
       'DP_ELABORACAO', 'SEC_ELABORACAO', 'DEP_ELABORACAO', 'DATA_FATO',
       'HORA_FATO', 'DESC_TIPOLOCAL', 'LOGRADOURO', 'NUMERO_LOGRADOURO',
       'LATITUDE', 'LONGITUDE', 'TIPO_PESSOA', 'SEXO_PESSOA', 'IDADE_PESSOA',
       'DATA_NASCIMENTO_PESSOA', 'COR_PELE', 'PROFISSAO', 'NATUREZA_APURADA'],
      dtype='object')

In [ ]:
# Realizando a troca de algumas colunas
dfsp_policia.rename(columns={'DP_CIRCUNSCRICAO ':'DP_CIRCUNSCRICAO'}, inplace=True)

In [ ]:
# Verificando a quantidade de ocorrências por ano
dfsp_policia.groupby(['NATUREZA_APURADA', 'ANO ESTATISTICA'],dropna=False).size().sort_values(ascending=False)

NATUREZA_APURADA                          ANO ESTATISTICA
MORTE DECORRENTE DE INTERVENÇÃO POLICIAL  2017               941
                                          2019               867
                                          2018               851
                                          2020               814
                                          2021               570
dtype: int64

##💿 **4 - Upload dos arquivos originais no Data Lake**

###⬆️ **Envio ao Google Cloud Storage**

In [ ]:
# Gerando arquivos em memória

# Homicidio doloso
dfsp_homicidio.to_excel('homicidios_Sp_orig.xlsx', index=False)

# Lesão Corporal
dfsp_lesao.to_excel('lesao_Sp_orig.xlsx', index=False)

# Latrocinio
dfsp_latrocinio.to_excel('latrocinio_Sp_orig.xlsx', index=False)

# Com intervenção policial
dfsp_policia.to_excel('policia_Sp_orig.xlsx', index=False)

In [ ]:
# Função para fazer upload de arquivo no bucket
def upload_blob(bucket, arquivo, destino):
    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(destino)

    blob.upload_from_filename(arquivo)

    print(
        f"Arquivo {arquivo} enviado para {destino}."
    )

# Salvando variáveis necessárias 
bucket = 'projeto-final-agsw'
arquivo = ['/content/homicidios_Sp_orig.xlsx','/content/lesao_Sp_orig.xlsx','/content/policia_Sp_orig.xlsx', '/content/latrocinio_Sp_orig.xlsx']
destino = ['originais/homicidios_Sp_orig.xlsx', 'originais/lesao_Sp_orig.xlsx', 'originais/policia_Sp_orig.xlsx', 'originais/latrocinio_Sp_orig.xlsx']
y = 0

# Realizando upload do arquivo
for x in arquivo:
  upload_blob(bucket, x, destino[y])
  y +=1


Arquivo /content/homicidios_Sp_orig.xlsx enviado para originais/homicidios_Sp_orig.xlsx.
Arquivo /content/lesao_Sp_orig.xlsx enviado para originais/lesao_Sp_orig.xlsx.
Arquivo /content/policia_Sp_orig.xlsx enviado para originais/policia_Sp_orig.xlsx.
Arquivo /content/latrocinio_Sp_orig.xlsx enviado para originais/latrocinio_Sp_orig.xlsx.


###⬆️ **Envio ao Cloud SQL (MySQL)**

In [ ]:
# Indicando parametros de conexão
INSTANCE_CONNECTION_NAME = f"link do servidor"
print(f"Você se conectou a instância: {INSTANCE_CONNECTION_NAME}")
DB_USER = "usuário"
DB_PASS = "senha"
DB_NAME = "dados-originais"

Você se conectou a instância: projeto-final-373521:southamerica-east1:mindfull-crime


In [ ]:
# iniciando variável de conexão
connector = Connector()

# Gerando função de retorno da conexão
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# Criando a conexão
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

In [ ]:
# Enviando para o MYSQL

# DF lesão
dfsp_homicidio.to_sql('dfsp_homicidio_orig', con=pool)

# DF lesão
dfsp_lesao.to_sql('dfsp_lesao_orig', con=pool)

# DF latrocinio
dfsp_latrocinio.to_sql('dfsp_latrocinio_orig', con=pool)

# DF policia
dfsp_policia.to_sql('dfsp_policia_orig', con=pool)

In [ ]:
# Utilizando sparkSQL para verificar se as tabelas estão no banco 
pool.connect().execute("SHOW TABLES;").fetchall()

[('dfsp_homicidio_orig',),
 ('dfsp_latrocinio_orig',),
 ('dfsp_lesao_orig',),
 ('dfsp_policia_orig',)]

In [ ]:
# Para limpar a conexão
connector.close()